### 1. Advanced CRUD Operations
- **Exercise 1**: Use `bulk_write` to insert, update, and delete multiple documents simultaneously.
- **Task 1**: Insert a dataset of `courses` where each course has multiple student enrollments, then display the data.

In [7]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),
    
    # Pelajaran baru yang ditambahkan
    InsertOne({'course': 'Biology 203', 'enrollments': 28, 'department': 'Biology'}),

    # Update pelajaran CS 102
    UpdateOne(
        {'course': 'CS 102'},
        {'$set': {'enrollments': 32}}
    ),

    # Hapus pelajaran 
    DeleteOne({'enrollments': {'$lt': 18}})
]

result = courses_collection.bulk_write(operations)
print("Bulk write successfully.")
print(f"Dokumen disisipkan: {result.inserted_count}")
print(f"Dokumen diperbarui: {result.modified_count}")
print(f"Dokumen dihapus: {result.deleted_count}")

print("Data semua course saat ini")
for course in courses_collection.find():
    print(course)

Bulk write successfully.
Dokumen disisipkan: 5
Dokumen diperbarui: 1
Dokumen dihapus: 1
Data semua course saat ini
{'_id': ObjectId('69118e8882cd7cf06038c2ec'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('69118e8882cd7cf06038c2ed'), 'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science'}
{'_id': ObjectId('69118e8882cd7cf06038c2ee'), 'course': 'History 201', 'enrollments': 20, 'department': 'History'}
{'_id': ObjectId('69118e8882cd7cf06038c2f0'), 'course': 'Biology 203', 'enrollments': 28, 'department': 'Biology'}


### 2. Complex Filtering and Querying

- **Exercise 2**: Filter courses with enrollments over 20 students.

In [2]:
# Query for courses with enrollments greater than 20
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

{'_id': ObjectId('6911275558c465f44c787edc'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('6911275558c465f44c787edd'), 'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science'}
{'_id': ObjectId('6911275558c465f44c787ee0'), 'course': 'Biology 203', 'enrollments': 28, 'department': 'Biology'}


- **Task 2**: Retrieve and display courses only from the 'Computer Science' or 'Mathematics' departments.

In [5]:
# Query courses in Computer Science or Mathematics departments
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

{'_id': ObjectId('6911275558c465f44c787edc'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('6911275558c465f44c787edd'), 'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science'}


### 3. Aggregation Framework for Data Analysis

- **Exercise 3**: Use aggregation to find the average enrollment per department.

In [6]:
# Average enrollment per department using aggregation
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Mathematics', 'average_enrollment': 30.0}
{'_id': 'History', 'average_enrollment': 20.0}
{'_id': 'Computer Science', 'average_enrollment': 32.0}
{'_id': 'Biology', 'average_enrollment': 28.0}


- **Task 3**: Create an aggregation pipeline that finds the maximum enrollment for each department.

In [7]:
# Maximum enrollment per department
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Mathematics', 'max_enrollment': 30}
{'_id': 'History', 'max_enrollment': 20}
{'_id': 'Computer Science', 'max_enrollment': 32}
{'_id': 'Biology', 'max_enrollment': 28}


### 4. Data Transformation using $project and $addFields

- **Exercise 4**: Use `$project` to rename and only show fields: `course_name`, `department_name`, and `enrollments`.

In [8]:
# Projection to rename fields
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('6911275558c465f44c787edc'), 'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'_id': ObjectId('6911275558c465f44c787edd'), 'enrollments': 32, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'_id': ObjectId('6911275558c465f44c787ede'), 'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}
{'_id': ObjectId('6911275558c465f44c787ee0'), 'enrollments': 28, 'course_name': 'Biology 203', 'department_name': 'Biology'}


- **Task 4**: Use `$addFields` to create a new field `enrollment_category` where enrollments > 20 are 'high' and others 'low'.

In [9]:
# Adding enrollment category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('6911275558c465f44c787edc'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('6911275558c465f44c787edd'), 'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('6911275558c465f44c787ede'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}
{'_id': ObjectId('6911275558c465f44c787ee0'), 'course': 'Biology 203', 'enrollments': 28, 'department': 'Biology', 'enrollment_category': 'high'}


### Homework for Students

- **Homework 1**: Perform an aggregation to get a count of courses per department.

In [5]:
pipeline = [
    {'$group': {'_id': '$department', 'course_count': {'$sum': 1}}}
]

print("Homework 1 - Count of courses per department:")
for result in courses_collection.aggregate(pipeline):
    print(result)

Homework 1 - Count of courses per department:
{'_id': 'Computer Science', 'course_count': 3}
{'_id': 'History', 'course_count': 3}
{'_id': 'Biology', 'course_count': 3}
{'_id': 'Mathematics', 'course_count': 3}


- **Homework 2**: Use `$match` and `$group` together to filter and get only courses with enrollments over 25 in 'Computer Science'.

In [8]:
pipeline = [
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gt': 25}}},
    {'$group': {'_id': '$department', 'total_courses': {'$sum': 1}, 'avg_enrollments': {'$avg': '$enrollments'}}}
]

print("Homework 2 - filter and get only courses with enrollments over 25 in 'Computer Science':")
for result in courses_collection.aggregate(pipeline):
    print(f"Department: {result['_id']}, Total Courses: {result['total_courses']}, Average Enrollments: {result['avg_enrollments']:.2f}")

Homework 2 - filter and get only courses with enrollments over 25 in 'Computer Science':
Department: Computer Science, Total Courses: 1, Average Enrollments: 32.00


- **Homework 3**: Apply `$lookup` to join `courses` collection with `students` collection based on student enrollments.

In [9]:
# Pembuatan collection students
from pymongo import MongoClient, InsertOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
students_collection = db['students']

# ====== Insert Data Mahasiswa ======
operations = [
    InsertOne({'student_id': 1, 'name': 'Ridwan', 'course': 'Math 101'}),
    InsertOne({'student_id': 2, 'name': 'Piaa', 'course': 'CS 102'}),
    InsertOne({'student_id': 3, 'name': 'Fauzan', 'course': 'History 201'}),
    InsertOne({'student_id': 4, 'name': 'Ilham', 'course': 'CS 102'}),
    InsertOne({'student_id': 5, 'name': 'Rasyad', 'course': 'Biology 203'})
]

result = students_collection.bulk_write(operations)

print("Students collection created successfully.")
print(f"Dokumen disisipkan: {result.inserted_count}")
print("\nData semua mahasiswa:")
for student in students_collection.find():
    print(student)

Students collection created successfully.
Dokumen disisipkan: 5

Data semua mahasiswa:
{'_id': ObjectId('6911949a82cd7cf06038c2f2'), 'student_id': 1, 'name': 'Ridwan', 'course': 'Math 101'}
{'_id': ObjectId('6911949a82cd7cf06038c2f3'), 'student_id': 2, 'name': 'Piaa', 'course': 'CS 102'}
{'_id': ObjectId('6911949a82cd7cf06038c2f4'), 'student_id': 3, 'name': 'Fauzan', 'course': 'History 201'}
{'_id': ObjectId('6911949a82cd7cf06038c2f5'), 'student_id': 4, 'name': 'Ilham', 'course': 'CS 102'}
{'_id': ObjectId('6911949a82cd7cf06038c2f6'), 'student_id': 5, 'name': 'Rasyad', 'course': 'Biology 203'}


In [10]:
# Homework 3
pipeline = [
    {
        '$lookup': {
            'from': 'students',        # Koleksi yang ingin digabungkan
            'localField': 'course',    # Field di koleksi 'courses'
            'foreignField': 'course',  # Field di koleksi 'students'
            'as': 'student_list'       # Hasil join disimpan dalam array ini
        }
    },
    {
        '$project': {                 # Pilih field yang akan ditampilkan
            '_id': 0,
            'course': 1,
            'department': 1,
            'enrollments': 1,
            'student_list.name': 1
        }
    }
]

# Jalankan agregasi 
print("Hasil Join Courses ↔ Students:\n")
for result in courses_collection.aggregate(pipeline):
    print(result)

Hasil Join Courses ↔ Students:

{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_list': [{'name': 'Ridwan'}]}
{'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science', 'student_list': [{'name': 'Piaa'}, {'name': 'Ilham'}]}
{'course': 'History 201', 'enrollments': 20, 'department': 'History', 'student_list': [{'name': 'Fauzan'}]}
{'course': 'Biology 203', 'enrollments': 28, 'department': 'Biology', 'student_list': [{'name': 'Rasyad'}]}
